# ColabSeg: Interactive Membrane Segmentation and Analysis

In [2]:
from colabseg.tensorvoting_wrapper import *
from colabseg.segmentation_gui import *
from colabseg.membrainseg_wrapper import generate_membrainseg_gui

# 1) Tensor Voting / MemBrain-seg

Run one of the following cells to perform either TomoSegMemTV or MemBrain-seg to segment the membranes in your tomogram:

In [2]:
generate_tensor_voting_gui() # run TomoSegMemTV

NameError: name 'generate_tensor_voting_gui' is not defined

In [1]:
generate_membrainseg_gui() # run MemBrain-seg

NameError: name 'generate_membrainseg_gui' is not defined

In [9]:
generate_tensor_voting_gui()

Button(description='Run TV Pipeline', style=ButtonStyle())

Button(description='Set optmized default', style=ButtonStyle())

# 2) Load Segmented MRC and Convert to Point Cloud

In [13]:
gui = JupyterFramework()
gui.gui_elements_loading()

  0%|          | 0/1 [00:00<?, ?it/s]

# 3) Visualize & Edit Segmentation 3D

In [14]:
gui.boot_gui()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

<Figure size 1200x800 with 0 Axes>

In [10]:
from colabseg.utilities import *

In [22]:
positions = create_sphere_points(1,1,1,1)

In [23]:
def write_xyz(point_cloud, output_filename):
    """Write an XYZ file for viewing in VMD"""
    xyz_file = "{}\n".format(len(np.asarray(point_cloud)))
    xyz_file += "pointcloud as xyz positions\n"
    for position in np.asarray(point_cloud):
        xyz_file += "C {} {} {}\n".format(position[0], position[1], position[2])
    text_file = open("{}".format(output_filename), "w")
    text_file.write(xyz_file)
    text_file.close()
    return

In [4]:
write_xyz(positions,"sphere_test_position_111.txt")

NameError: name 'write_xyz' is not defined

In [20]:
import MDAnalysis as mda
import numpy as np
import math

In [25]:

def lstsq_sphere_fitting(positions):
    """
    lstsq_sphere_fitting: fit a given set of 3D points (x, y, z) to a sphere.
    Args:
       positions: a two dimentional numpy array, of which each row represents (x, y, z) coordinates of a point
    Returns:
       radius, x0, y0, z0
    Source: https://gist.github.com/WuyangLI/4bf4b067fed46789352d5019af1c11b2
    """
    # add column of ones to pos_xyz to construct matrix A
    pos_xyz = positions
    row_num = pos_xyz.shape[0]
    A = np.ones((row_num, 4))
    A[:,0:3] = pos_xyz

    # construct vector f
    f = np.sum(np.multiply(pos_xyz, pos_xyz), axis=1)

    sol, residules, rank, singval = np.linalg.lstsq(A,f)

    # solve the radius
    radius = math.sqrt((sol[0]*sol[0]/4.0)+(sol[1]*sol[1]/4.0)+(sol[2]*sol[2]/4.0)+sol[3])

    return radius, sol[0]/2.0, sol[1]/2.0, sol[2]/2.0

In [40]:
pos = np.loadtxt("missing_wedge_sphere.txt")
r, x0, y0, z0 = lstsq_sphere_fitting(pos)

/anaconda3/envs/mesh/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


In [57]:
np.testing.assert_almost_equal(x0, 1)

AssertionError: 
Arrays are not almost equal to 7 decimals
 ACTUAL: 0.0
 DESIRED: 1